In [1]:
import csbgnpy.pd.io.cd
import csbgnpy.pd.io.sbgnml
import sbgn2an.utils
import sbgn2an.ini

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge = csbgnpy.pd.io.sbgnml.read("generated/MERGE.sbgnml")

## Initial state

We compute all initial states that respect the constraints given in the paper, and that contains the pRB:E2F1:DP2 complex.

In [3]:
inis = sbgn2an.ini.get_ini(merge, in_ini=[merge.get_entity("Complex([SubMacromolecule([][@|@|@|@]pRB*)|SubMacromolecule([][@]DP1*)|SubMacromolecule([][@Ser364|@Ser31|@Ser403|@Lys|@]E2F1)][][]pRB*/E2F1/DP1*#Compartment(nucleoplasm))", by_string=True)])
print(len(inis))

24


We obtain 24 possible initial states.
We check the difference between each initial state.
For this we first compute the intersection of all initial states.

In [4]:
inter = set([e.id for e in inis[0]])
for ini in inis[1:]:
    inter = inter.intersection(set([e.id for e in ini]))

In [5]:
print([len(ini) for ini in inis])
print(len(inter))

[106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106]
102


Each of the 24 initial states is constituted of 104 entities, 100 of which are common to all initial states.
Hence we check the nature of the entities that do not belong to all initial states.

In [6]:
tot_diff = set([])
for ini in inis:
    ini = set([e.id for e in ini])
    diff = ini.difference(inter)
    tot_diff = tot_diff.union(diff)
    
print([str(merge.get_entity(e, by_id=True)) for e in tot_diff])


['SimpleChemical([][]NAM#Compartment([]nucleoplasm))', 'SimpleChemical([][]NAD+#Compartment([]nucleoplasm))', 'Macromolecule([][@|@|@Ser15|@Ser20]p53#Compartment([]nucleoplasm))', 'Macromolecule([][]E2F4#Compartment([]nucleoplasm))', 'Macromolecule([][]DP2*#Compartment([]cytoplasm))', 'SimpleChemical([][]NMN#Compartment([]nucleoplasm))', 'Macromolecule([][]E2F4#Compartment([]cytoplasm))', 'Macromolecule([][@|@|@Ser15|@Ser20]p53#Compartment([]cytoplasm))', 'Macromolecule([][]DP2*#Compartment([]nucleoplasm))']


For each initial state, the 4 entities that do not belong to all initial states are to be chosen as follows (2 x 2 x 2 x 3 = 24 possibilities): 

* p53: "Macromolecule([][@|@|@Ser15|@Ser20]p53#Compartment([]cytoplasm))" or "Macromolecule([][@|@|@Ser15|@Ser20]p53#Compartment([]nucleoplasm))"
* E2F4: "Macromolecule([][]E2F4#Compartment([]cytoplasm))" or "Macromolecule([][]E2F4#Compartment([]nucleoplasm))"
* DP2: "Macromolecule([][]DP2\*#Compartment([]cytoplasm))" or "Macromolecule([][]DP2*#Compartment([]nucleoplasm))"
* NAD+/NAM/NMN: "SimpleChemical([][]NAD+#Compartment([]nucleoplasm))" or "SimpleChemical([][]NAM#Compartment([]nucleoplasm))" or "SimpleChemical([][]NMN#Compartment([]nucleoplasm))"

For p53, E2F4, DP2, the entities have two localizations (cytoplasm and nucleoplasm) and cycling translocation processes from one localization to the other.
For the NAD+/NAM/NMN entities, there is a cycle producing the three entities (NAD+->NAM->NMN->NAD+).

We choose the initial state with the p53, E2F4 and DP2 entities located in the cytoplasm, and with the NAD+ entity.

In [7]:
p53 = "Macromolecule([][@|@|@Ser15|@Ser20]p53#Compartment([]cytoplasm))"
e2f4 = "Macromolecule([][]E2F4#Compartment([]cytoplasm))"
dp2 = "Macromolecule([][]DP2*#Compartment([]cytoplasm))"
nad = "SimpleChemical([][]NAD+#Compartment([]nucleoplasm))"

final_inis = []
for ini in inis:
    if merge.get_entity(p53, by_string=True) in ini and merge.get_entity(e2f4, by_string=True) in ini and merge.get_entity(dp2, by_string=True) in ini and merge.get_entity(nad, by_string=True) in ini:
        final_inis.append(ini)
        
print(len(final_inis))
ini = final_inis[0]
%store ini

1
Stored 'ini' (list)
